# Day 1-00.04: EXAONE 모델 준비하기

EXAONE-3.5-2.4B-Instruct 모델을 4bit + LoRA 구성으로 빠르게 준비합니다.


### 1. 라이브러리 로드 및 bitsandbytes 확인

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

print("✅ 라이브러리 로드 완료")


> **bitsandbytes 4bit 준비**
> - 기본 런타임에 bitsandbytes가 설치돼 있어야 합니다.
> - 설치되어 있지 않은 경우 `pip install -q bitsandbytes accelerate` 실행 후 런타임을 재시작하세요.


1. 그냥 (FP16/FP32) 모델

일반적으로 Hugging Face에서 불러오는 모델은 FP16(half precision) 또는 **FP32(full precision)**로 저장돼 있습니다.

모든 가중치(weight) 값이 16bit 또는 32bit 부동소수점으로 표현돼서 정확도가 가장 높고 원래 성능을 유지합니다.

하지만 GPU 메모리를 많이 차지하고, 연산량도 큽니다.

2. 4비트 퀀타이즈 모델

32bit → 4bit로 줄여서 메모리 사용량을 약 8배 줄임.

예: 13B 파라미터 모델

FP16: 약 26GB 필요

4bit: 약 6.5GB 필요

대신 가중치 표현 정밀도가 낮아져서 정보 손실이 생깁니다.

추론 속도도 빨라지고, 적은 GPU에서도 돌릴 수 있지만,

**출력 품질(정확도)**이 조금 떨어질 수 있어요. (보통 1~3% 정도 성능 저하 보고됨)

3. 실제 출력 차이

FP16/32 모델: 원래 학습된 지식을 가장 충실히 반영.

4bit 모델: 의미는 비슷하게 내지만,

문맥 유지가 짧아진다거나,

세밀한 수치/희귀 단어에서 에러가 날 수 있음.

큰 맥락에서는 차이 안 보일 수 있지만, 어려운 질문일수록 차이가 두드러집니다.

In [ ]:
def require_bitsandbytes(compute_dtype=torch.float16) -> BitsAndBytesConfig:
    try:
        import bitsandbytes as bnb  # noqa: F401
        print("✅ bitsandbytes가 설치되어 있습니다.")
    except ImportError as exc:
        raise RuntimeError(
            "bitsandbytes가 설치되어 있지 않습니다. `pip install -q bitsandbytes accelerate` 실행 후 런타임을 재시작하세요."
        ) from exc

    try:
        config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
        )
        print("✅ 4bit 양자화 구성이 완료되었습니다.")
        return config
    except Exception as exc:
        raise RuntimeError(
            "bitsandbytes 초기화 중 오류가 발생했습니다. `pip install -U bitsandbytes` 실행 후 런타임을 재시작한 뒤 다시 시도하세요."
            f"오류 메시지: {exc}"
        ) from exc


### 2. 모델과 토크나이저 로드
- Hugging Face에서 EXAONE 모델과 토크나이저를 불러옵니다.
- 4bit quantization을 적용해 메모리를 줄입니다.


In [ ]:
MODEL_NAME = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
print(f"📥 모델 다운로드: {MODEL_NAME}")

quant_config = require_bitsandbytes(compute_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    padding_side="right",
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    quantization_config=quant_config,
)

print("✅ 모델과 토크나이저 로드 완료")
print(f"   - 패딩 토큰: {tokenizer.pad_token}")
print(f"   - 디바이스: {next(model.parameters()).device}")


### 3. LoRA 설정 및 적용
- 4bit 양자화로 기본 모델의 메모리 사용량을 줄여 학습 준비를 합니다.
- LoRA는 **추가 학습을 위한 어댑터**로, 전체 가중치를 건드리지 않고 일부 모듈(q/k/v/out)에만 작은 행렬을 붙입니다.
- 이 노트북에서는 훈련을 실행하지 않고 **어댑터를 붙여 사용할 준비 상태**까지 만들며, 이후 파인튜닝 노트북에서 실제 학습이 진행됩니다.


In [ ]:
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"],
    bias="none",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print("✅ LoRA 적용 완료")
print(f"   - 학습 가능한 파라미터: {trainable_params:,}")
print(f"   - 전체 파라미터: {total_params:,}")
print(f"   - 학습 비율: {trainable_params / total_params:.3%}")


### 4. 간단한 생성 테스트
LoRA 어댑터를 붙인 모델이 정상적으로 동작하는지 확인합니다. (아직 학습은 하지 않았으므로, 출력은 기본 모델과 비슷합니다.)


In [ ]:
TEST_PROMPT = "질문: 인공지능이란 무엇인가요?답변:"
inputs = tokenizer(TEST_PROMPT, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=60,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("🧪 예시 출력")
print(answer)


### 5. 다음 단계로
- 준비된 모델과 토크나이저를 저장하거나 바로 이어지는 파인튜닝 실습에 사용할 수 있습니다.
- 다음 노트북: `main-practice/03_fine_tuning_with_lora.ipynb`
